In [ ]:
from matplotlib.pyplot import *
import numpy as np
import pandas as pd
import os
import importlib

from ATARI.sammy_interface import sammy_classes, sammy_functions, template_creator

from ATARI.ModelData.particle_pair import Particle_Pair
from ATARI.ModelData.experimental_model import Experimental_Model
from ATARI.utils import atario
from ATARI.utils import plotting as myplot
from ATARI.AutoFit.functions import *

from copy import copy


In [ ]:
sammypath = ''
assert(sammypath != '')

In [ ]:
%matplotlib widget

## Measurement Data

The following measurement data are imported from the work of Jesse Brown at RPI (currently at ORNL).
These measurements are detailed in https://www.tandfonline.com/doi/full/10.1080/00295639.2023.2249786 and consist of 6 experimental configurations.
Of the 4 transmission measurements only 3 are used for evaluation and the 4th (45m FP) is used for validation. 
Both 2 capture measurements are used in evaluation.

The transmission data are reliable in the range 0.15-100 keV.
The capture data are reliable from 0.15-200 keV. 

The Syndat example notebooks detail how synthetic data models are built for each of these measurements. In this notebook, the AutoFit module will be used to fit the actual data. However, the experimental models developed for the synthetic data are used here as well. These are loaded from the save syndat control instance.



In [ ]:
# load the syndat control instance
syndat_control = atario.load_general_object(os.path.realpath("./results/SyndatControl_E150_5e3.pkl"))

# redefine template paths relative to this location
syndat_control.redefine_exp_template_directory(os.path.realpath("./data"))

experimental_titles = []
experimental_models = []
### unpack syndat models and save into zipped lists
for each in syndat_control.syndat_models:
    if each.title == "trans1mm":
        exptrans1 = each.generative_experimental_model
    elif each.title == "trans3mm":
        exptrans3 = each.generative_experimental_model
    elif each.title == "trans6mm":
        exptrans6 = each.generative_experimental_model
    elif each.title == "cap1mm":
        expcap1 = each.generative_experimental_model
    elif each.title == "cap2mm":
        expcap2 = each.generative_experimental_model
    else:
        raise ValueError("Unknown Title")
    
    print(each.title)
    experimental_titles.append(each.title)
    experimental_models.append(each.generative_experimental_model)

In [ ]:
from ATARI.sammy_interface.sammy_io import read_idc

### 1mm capture data
capdat1 = sammy_functions.readlst(os.path.realpath("./data/yield_ta1b_unsmooth.dat"))
capdat1 = capdat1.loc[(capdat1.E<max(expcap1.energy_range)) & (capdat1.E>min(expcap1.energy_range)), :]
### 2mm capture data
capdat2 = sammy_functions.readlst(os.path.realpath("./data/yield_ta2_unsmooth.dat"))
capdat2 = capdat2.loc[(capdat2.E<max(expcap2.energy_range)) & (capdat2.E>min(expcap2.energy_range)), :]

### 1mm Transmission data
transdat1 = sammy_functions.readlst(os.path.realpath("./data/trans-Ta-1mm.twenty"))
transdat1_cov = read_idc(os.path.realpath('./data/trans-Ta-1mm.idc'))
transdat1 = transdat1.loc[(transdat1.E<max(exptrans1.energy_range)) & (transdat1.E>min(exptrans1.energy_range)), :]
### 3mm transmission data
transdat3 = sammy_functions.readlst(os.path.realpath("./data/trans-Ta-3mm.twenty"))
transdat3_cov = read_idc(os.path.realpath("./data/trans-Ta-3mm.idc"))
transdat3 = transdat3.loc[(transdat3.E<max(exptrans3.energy_range)) & (transdat3.E>min(exptrans3.energy_range)), :]
### 6mm transmission data
transdat6 = sammy_functions.readlst(os.path.realpath("./data/trans-Ta-6mm.twenty"))
transdat6_cov = read_idc(os.path.realpath("./data/trans-Ta-6mm.idc"))
transdat6 = transdat6.loc[(transdat6.E<max(exptrans6.energy_range)) & (transdat6.E>min(exptrans6.energy_range)), :]

In [ ]:
from ATARI.utils.datacontainers import Evaluation_Data, Evaluation

### setup evaluation data class - it has handy methods
eval_data = Evaluation_Data(
    experimental_titles=experimental_titles,
    experimental_models=experimental_models,
    datasets=[transdat1, transdat3, transdat6, capdat1, capdat2], # make sure these zipped lists align with experiment model lists
    covariance_data = [transdat1_cov, transdat3_cov, transdat6_cov, {}, {}], # make sure these zipped lists align with experiment model lists
)

### truncate the evaluation to a window of interest
energy_range_all = (1000,1030)
eval_data = eval_data.truncate(energy_range_all)

### Fit from JEFF prior
This approach interfaces directly with run_sammy_YW to simultaneously fit all datasets starting from the specified parameters read from the JEFF-3.3 evaluation. Later in this notebook, the AutoFit module will be used to fit the data without a prior. Note that AutoFit also uses run_sammy_YW `under-the-hood'.

In [ ]:
sammyRTO = sammy_classes.SammyRunTimeOptions(sammypath,
                             **{"Print"   :   True,
                              "bayes"   :   True,
                              "keep_runDIR"     : False
                              })

## load, truncate, and set vary on the evaluated ladder
jeff_file = os.path.realpath("./data/73-Ta-181g.jeff33")
eval_jeff = Evaluation.from_ENDF6("jeff", 7328, jeff_file, sammyRTO)
eval_jeff = eval_jeff.truncate(energy_range_all, external_resonance_energy_buffer=5)
eval_jeff.update_vary_on_resonance_ladder(varyE=1, varyGg=0, varyGn1=1)


### sammy input YW
sammyINPyw = sammy_classes.SammyInputDataYW(
    particle_pair = syndat_control.particle_pair,
    resonance_ladder = eval_jeff.resonance_ladder,  

    datasets= eval_data.datasets,
    experiments = eval_data.experimental_models,
    experimental_covariance= eval_data.covariance_data, 
    
    max_steps = 10,
    iterations = 3,
    step_threshold = 0.01,
    autoelim_threshold = None,

    LS = False,
    LevMar = True,
    LevMarV = 2,
    LevMarVd= 5,
    maxF= 1.5,
    initial_parameter_uncertainty = 0.01
    )


In [ ]:
sammyOUT_JEFF = sammy_functions.run_sammy_YW(sammyINPyw, sammyRTO)

In [ ]:
importlib.reload(myplot)
fig = myplot.plot_reduced_data_TY(eval_data.datasets, 
                                  eval_data.experimental_models,
                                  fits = sammyOUT_JEFF.pw_post,
                                  xlim=(energy_range_all))

fig.tight_layout()

### Fit and eliminate from initial feature bank instead of prior

The fit and eliminate module (imported as fe) performs the non-linear optimization and stepwise elimination to some target number of resonances. There are two primary methods: fe.initial_fit and fe.eliminate. Both methods take a solver and a resonance ladder as an argument. 

The starting resonance ladder for fe.initial_fit can be generated with the initial feature bank utilities below while the output resonance ladder from fe.initial_fit needs to be parsed and supplied to fe.eliminate.

In [ ]:
from ATARI.AutoFit.initial_FB_solve import InitialFBOPT
import ATARI.AutoFit.functions as fn

initialFBopt = InitialFBOPT(starting_Gn1_multiplier = 100,
                            starting_Gg_multiplier = 1,
                            fit_all_spin_groups=False,
                            spin_group_keys = ['3.0'],
                            num_Elam = 15,
                            )

### The initial feature bank can be designed to include features that the evaluator wants, i.e., fixed external resonances
initial_resonance_ladder, external_resonance_ladder = fn.get_initial_resonance_ladder(initialFBopt, syndat_control.particle_pair, energy_range_all, external_resonance_ladder=None)
print(external_resonance_ladder.E)
initial_resonance_ladder

The solver class describes how the non-linear optimization will be solved and has several options both internal and external to SAMMY. It also contains much of the inputs needed for different run_sammy() functions and stores them in a convenient way. The fit method is linked to the appropriate run_sammy upon instantiation, creating a uniform way to fit a ladder that is used throughout fit and eliminate. This allows different solver instances to be used with the same fe code.

An options instance is created and passed to the instantiation of the solver class itself, the solver class is then passed to fe.

In [ ]:
from ATARI.sammy_interface.sammy_classes import SolverOPTs_YW
from ATARI.AutoFit import fit_and_eliminate, sammy_interface_bindings, auto_fit

solver_options = SolverOPTs_YW(max_steps = 20,
                            step_threshold=0.01,
                            LevMar=True, LevMarV=1.5,LevMarVd=5,minF = 1e-4,maxF = 1.5,
                            initial_parameter_uncertainty=0.1, iterations=1,
                            idc_at_theory=False)

solver_options_elim = SolverOPTs_YW(max_steps = 10,
                            step_threshold=0.01,
                            LevMar=True, LevMarV=1.5,LevMarVd=5,minF = 1e-4,maxF = 1.5,
                            initial_parameter_uncertainty=0.1, iterations=1,
                            idc_at_theory=False)


solver_initial = sammy_interface_bindings.Solver_factory(sammyRTO, solver_options._solver, solver_options, syndat_control.particle_pair, eval_data) 
solver_elim = sammy_interface_bindings.Solver_factory(sammyRTO, solver_options._solver, solver_options, syndat_control.particle_pair, eval_data)

In [ ]:
# An example of how the solver class can be used directly, fitting from the JEFF prior resonance ladder
sammyOUT_JEFF = solver_initial.fit(sammyINPyw.resonance_ladder)

In [ ]:
import ATARI.utils.plotting as myplot

fig = myplot.plot_reduced_data_TY(eval_data.datasets,
                            eval_data.experimental_models,
                            priors = sammyOUT_JEFF.pw,
                            fits=sammyOUT_JEFF.pw_post,
                            
                            xlim=energy_range_all)
fig.tight_layout()

The fit_and_eliminate module has and option class and a utility class. An instance of options is created and passed to create and instance of the utility class along with the solver instances we created before. The fe.initial_fit and fe.eliminate methods are then demonstrated. An important input to these methods is the fixed_resonance_ladder

In [ ]:

#### Fit and eliminate
fit_and_elim_options = fit_and_eliminate.FitAndEliminateOPT(chi2_allowed=0.01,
                                                            width_elimination=False, 
                                                            greedy_mode=True,
                                                            deep_fit_max_iter = 10,
                                                            deep_fit_step_thr = 0.01,
                                                            LevMarV0_priorpassed = 0.1)

fe = fit_and_eliminate.FitAndEliminate(solver_initial=solver_initial, solver_eliminate=solver_elim, options=fit_and_elim_options, particle_pair=syndat_control.particle_pair)


initial_samout = fe.initial_fit(initial_resonance_ladder, fixed_resonance_ladder=external_resonance_ladder)

initial_feature_bank, fixed_resonance_ladder = separate_external_resonance_ladder(initial_samout.par_post, fe.output.external_resonance_indices)

elimination_history = fe.eliminate(initial_feature_bank, target_ires=0, fixed_resonance_ladder=fixed_resonance_ladder, )

print(elimination_history[4]['selected_ladder_chars'].par_post)

print(elimination_history[0]['selected_ladder_chars'].par_post)

In [ ]:
import ATARI.utils.plotting as myplot

fig = myplot.plot_reduced_data_TY(eval_data.datasets,
                            eval_data.experimental_models,
                            priors = fe.output.initial_fits[0].pw,
                            fits=elimination_history[5]['selected_ladder_chars'].pw_post,
                            
                            xlim=energy_range_all)
fig.tight_layout()

### Careful, the full autofit with CV takes about an hour, parallel CV cannot be used within a notebook


The AutoFit module does several things. 
The largest internal module is the fit_and_eliminate module which will is passed a starting resonance ladder and will fit it, then reduce complexity by eliminating resonances down to some target number of resonances. You must define solver options for both the initial fit and the elimination fits.

The starting resonance ladder is given by the initial feature bank class which also has a set of user defined options.

Currently, are two ways AutoFit determines that target 1) via a delta chi2 threshold that is a selected hyperparameter or 2) via a cross validation approach.


In [ ]:

autofit_options = auto_fit.AutoFitOPT(save_elimination_history   = True,
                                    save_CV_elimination_history  = False,
                                    parallel_CV                  = False,
                                    parallel_processes           = 5,
                                    final_fit_to_0_res           = False)

af = auto_fit.AutoFit(sammyRTO, syndat_control.particle_pair, solver_options, solver_options_elim, AutoFit_options=autofit_options, fit_and_elim_options=fit_and_elim_options)
autofit_out = af.fit(eval_data, initial_resonance_ladder, fixed_resonance_indices=fixed_resonance_ladder)